## Amazon Product Analysis, including Review Analysis ##
The .csv has been imported in the first project file to create a star schema image, attached in the folder. 

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///amazon_sales.csv')

csv_filepath = 'data/amazon_sales.csv'
raw_df = pd.read_csv(csv_filepath)

raw_df.to_sql('stg_amazon_products_reviews_raw', engine, if_exists='replace', index=False)

print("Raw data loaded successfully into the 'stg_amazon_products_reviews_raw' table.")

In [ ]:
%load_ext sql
%sql sqlite:///amazon_sales.csv

In [ ]:
%%sql

CREATE TABLE dim_product(
    product_key INT PRIMARY KEY,
    product_id VARCHAR(20) NOT NULL UNIQUE, 
    product_name VARCHAR(50) NOT NULL,
    category TEXT,
    discounted NUMERIC,
    actual_price NUMERIC,
    discount_percentage NUMERIC, 
    product_link LINK
);

In [ ]:
%%sql

INSERT INTO dim_product(
    product_id, product_name, category, actual_price, discount_percentage, product_link
    )
SELECT
    t1.product_id,
    t1.product_name,
    t1.category,
    t1.actual_price,
    t1.discount_percentage,
    t1.product_link
FROM (
    SELECT
        product_id,
        product_name,
        category, 
        actual_price,
        discount_percentage,
        product_link
    FROM stg_amazon_products_reviews_raw
    GROUP BY 1, 2
) AS t1;

In [ ]:
%%sql

CREATE TABLE dim_review_detail(
    review_key INT UNIQUE,
    review_content TEXT, 
    img_link LINK
)

In [ ]:
%%sql 

INSERT INTO dim_review_detail(
    review_content, img_link
)
SELECT
    T1.review_content,
    T1.img_link
FROM (
    SELECT 
        review_content,
        img_link
    FROM stg_amazon_products_reviews_raw
    GROUP BY 1, 2
) AS T1;

In [ ]:
%%sql 

CREATE TABLE dim_user(
    user_key INT UNIQUE,
    user_id VARCHAR(20) NOT NULL UNIQUE,
    user_name VARCHAR(50),
    customer_location TEXT
)

In [ ]:
%%sql

INSERT INTO dim_user(
    user_id, user_name
    )
SELECT
    T1.user_id,
    T1.user_name
FROM (
    SELECT
        user_id,
        user_name
    FROM stg_amazon_products_reviews_raw
    GROUP BY 1, 2
) AS T1;

In [ ]:
%%sql

CREATE TABLE fact_review_metrics(
    review_key INT PRIMARY KEY NOT NULL,
    date_key DATE NOT NULL,
    user_key VARCHAR(30) NOT NULL,
    product_key VARCHAR(30) NOT NULL,
    rating METRIC,
    rating_count METRIC,
    rating_title DESCRIPTIVE
);

In [ ]:
%config SqlMagic.style = 'plain'

In [ ]:
%sql sqlite:///amazon_sales.csv

In [19]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///amazon_sales.csv')

sql_query = """
SELECT 
    COUNT(*) AS total_rows, 
    COUNT(DISTINCT product_id) AS unique_products
FROM stg_amazon_products_reviews_raw
LIMIT 10;
"""

result_df = pd.read_sql(sql_query, engine)
result_df

,total_rows,unique_products
0,1465,1351
